This notebook contains a comparison of AgentBind to MPRA data from Tewhey et al. Cell 2016

In [3]:
%pylab inline

import pandas as pd
import scipy.stats
import sklearn.metrics
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

data = pd.read_csv("../mpra/Tewhey_MPRA_hg19.tab", sep="\t")
data["chrom"] = data["chrom"].apply(lambda x: "chr"+str(x))

Populating the interactive namespace from numpy and matplotlib


In [5]:
factors_str = "FOS STAT1 CEBPB JunD STAT3 RFX5 ETS1 NFYA CTCF EBF1 SP1 \
PU1 RUNX3 NFYB Nrf1 ELF1 NFKB TCF3 Mxi1 USF1 YY1 USF2 ZEB1 PAX5 POU2F2 \
NRSF PBX3 MEF2A E2F4 BHLHE40 ELK1 NFIC MEF2C Max SRF Znf143 IRF4 ZBTB33"
#factors_str = "FOS" # this line is for testing only
factors = factors_str.split()

model = "dnase-controlled"
#model = "baseline"
#model = "gc-controlled"

if model == "dnase-controlled":
    experiment = "AgentBind-GM12878-DanQ-trainon-DNase-GC-balanced"
if model == "gc-controlled":
    experiment = "AgentBind-GM12878-DanQ-unfixed-rnn-trans-GC-balanced"
if model == "baseline":
    experiment = "AgentBind-GM12878-DanQ-unfixed-rnn-trans"

chrom_ = []
pos_ = []
factor_ = []
score_ = []
rank_ = []
score_norm_ = []

version = "equal"

if version == "plus1":
    data["pos2"] = data["pos"]+1 # TODO figure out offset (so far -1 is best)
if version == "minus1":
    data["pos2"] = data["pos"]-1
if version == "equal":
    data["pos2"] = data["pos"]

def GetRank(vallist, val):
    slist = sorted(vallist)
    return slist.index(val)
    

for factor in factors:
    print (factor)
    position_line = True
    weight_file = "/storage/pandaman/project/%s/storage/AgentBind-GM12878-DanQ/tmp/%s+GM12878/seqs_one_hot_c/vis-weights-total/weight.txt" %(experiment, factor)
    for line in open(weight_file):
        if position_line:
            chromID, motif_start, motif_end, seq_start, seq_end, strand = line.strip().split(";")
            motif_start = int(motif_start)
            motif_end = int(motif_end)
            seq_start = int (seq_start)
            seq_end = int(seq_end)
            position_line = False
        else:
            weights = [float(elem) for elem in line.strip().split(";")]
            region_snps = list(data[(data["chrom"]==chromID) & (data["pos2"]>=seq_start) & (data["pos2"]<seq_end)]["pos2"].values)
            other_snps = []
            for item in region_snps:
                other_snps.append(item+1)
                other_snps.append(item-1)
            for pos in region_snps+other_snps:
                try:
                    if strand == "+":
                        score_pred = weights[pos - seq_start]
                    elif strand == "-":
                        score_pred = weights[1000 - (pos - seq_start) - 1]
                    else:
                        print("Incorrect symbol: %s" %strand)
                        continue
                except: continue
                score_rank = GetRank(weights, score_pred)
                # Add to list
                chrom_.append(chromID)
                pos_.append(pos)
                factor_.append(factor)
                score_.append(score_pred)
                score_norm_.append(score_pred/np.mean(weights))
                rank_.append(score_rank)
            position_line = True
            
gcam = pd.DataFrame({"chrom": chrom_, "rank": rank_, "pos2": pos_, "factor": factor_, "score": score_, "score.norm": score_norm_})
gcam.to_csv("gcam_scores_%s_%s.csv"%(version, model))

FOS
STAT1
CEBPB
JunD
STAT3
RFX5
ETS1
NFYA
CTCF
EBF1
SP1
PU1
RUNX3
NFYB
Nrf1
ELF1
NFKB
TCF3
Mxi1
USF1
YY1
USF2
ZEB1
PAX5
POU2F2
NRSF
PBX3
MEF2A
E2F4
BHLHE40
ELK1
NFIC
MEF2C
Max
SRF
Znf143
IRF4
ZBTB33


In [ ]:
# do analysis overall and by TF

factors_str = "FOS STAT1 CEBPB JunD STAT3 RFX5 ETS1 NFYA CTCF EBF1 SP1 \
PU1 RUNX3 NFYB Nrf1 ELF1 NFKB TCF3 Mxi1 USF1 YY1 USF2 ZEB1 PAX5 POU2F2 \
NRSF PBX3 MEF2A E2F4 BHLHE40 ELK1 NFIC MEF2C Max SRF Znf143 IRF4 ZBTB33"
#factors_str = "FOS" # this line is for testing only
factors = factors_str.split()

model = "dnase-controlled"
gcam = pd.read_csv("gcam_scores_equal_%s.csv"%model)

def GetBest(scores):
    best = 0 #scores.values[0]
    for item in scores:
        if item > best: best = item
    return best

data["pos2"] = data["pos"]+1


score_corr_ = []
score_corr_p_ = []
rank_corr_ = []
rank_corr_p_ = []
num_emvars_ = []
auc_score_ = []
auc_rank_ = []
auc_norm_ = []
auprc_score_ = []

FDRTHRESH = 0.05

aucinfo = {}
auprcinfo = {}

for factor in ["ALL"]+factors:
    if factor == "ALL":
        comp = pd.merge(data, gcam, on=["chrom","pos2"]).groupby(["chrom","pos"], as_index=False).agg(
            {"score": GetBest, "score.norm": np.max, "rank": np.max,
            "LogSkew.Comb": np.mean, "C.Skew.fdr": np.mean})
    else:
        comp = pd.merge(data, gcam[gcam["factor"]==factor], on=["chrom","pos2"]).groupby(["chrom","pos"], as_index=False).agg(
            {"score": GetBest, "score.norm": np.max, "rank": np.max,
            "LogSkew.Comb": np.mean, "C.Skew.fdr": np.mean})
    comp = comp[~np.isnan(comp["C.Skew.fdr"])] # restrict to "RegHits"
    comp["LogSkew.Comb.abs"] = comp["LogSkew.Comb"].apply(abs)
    comp = comp[comp["score"]>0] # remove things with weird gcam scores

    emv = comp[(comp["C.Skew.fdr"]>-1*np.log10(FDRTHRESH)) ] # log10?

    num_emvars_.append(emv.shape[0])

    if factor == "ALL":
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.scatter(emv["LogSkew.Comb.abs"], emv["score"].apply(abs))
        ax.set_ylim(bottom=-0.00005, top=0.001)
        ax.set_title(factor)
    xx = scipy.stats.spearmanr(emv["LogSkew.Comb.abs"], emv["score"].apply(abs))
    score_corr_.append(xx[0])
    score_corr_p_.append(xx[1])
    
    xx = (scipy.stats.spearmanr(emv["LogSkew.Comb.abs"], emv["rank"]))
    rank_corr_.append(xx[0])
    rank_corr_p_.append(xx[1])


    comp["emvar"] = (comp["C.Skew.fdr"]>-1*np.log10(FDRTHRESH))

    if comp.shape[0] > 5:
        if factor == "ALL":
            comp.boxplot(by="emvar", column="score", sym="")
            print(scipy.stats.mannwhitneyu(comp[comp["emvar"]]["score"],
                                           comp[~comp["emvar"]]["score"]))
            print(len(comp[comp["emvar"]]["score"]))
            print(len(comp[~comp["emvar"]]["score"]))

        # Get auroc
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(comp["emvar"], comp["score"], pos_label=True)
        auc_score_.append(sklearn.metrics.auc(fpr, tpr))
        aucinfo[factor] = (fpr, tpr)
        precision, recall, thresholds = sklearn.metrics.precision_recall_curve(comp["emvar"], comp["score"], pos_label=True)
        auprcinfo[factor] = (precision, recall)                                                                       
        auprc_score_.append(sklearn.metrics.average_precision_score(comp["emvar"], comp["score"], pos_label=True))
                                                                               
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(comp["emvar"], comp["rank"], pos_label=True)
        auc_rank_.append(sklearn.metrics.auc(fpr, tpr))
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(comp["emvar"], comp["score.norm"], pos_label=True)
        auc_norm_.append(sklearn.metrics.auc(fpr, tpr))

    else:
        auc_score_.append(float("nan"))
        auc_rank_.append(float("nan"))
        auc_norm_.append(float("nan"))
        auprc_score_.append(float("nan"))

res = pd.DataFrame({"factor": ["ALL"]+factors,
                   "r.score": score_corr_,
                   "r.score.p": score_corr_p_,
                   "r.rank": rank_corr_,
                   "r.rank.p": rank_corr_p_,
                   "num.emvar": num_emvars_,
                   "auc.score": auc_score_,
                   "auc.rank": auc_rank_,
                   "auc.norm": auc_norm_,
                   "auprc.score": auprc_score_})